### Blood segmenting per available information

In [1]:
import numpy as np
import cv2
import pandas as pd
import requests

from matplotlib import pyplot as plt

In [2]:
#Function to download file from internet url
def get_image(url):
    local_filename = 'blood.jpg'
    
    # NOTE the stream=True parameter below
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                # If you have chunk encoded response uncomment if
                # and set chunk_size parameter to None.
                #if chunk: 
                f.write(chunk)
    
    img = cv2.imread(local_filename)
    
    return img

In [3]:
def smear(xmin,ymin,xmax,ymax,img,type,color):
    if (xmin<2):
        xmin = 2
    if (ymin<2):
        ymin = 2

    tmp = np.ones([(ymax-ymin), 2, 3]) * 0
    img[ymin:ymax, (xmin-2):xmin, 0:3] = tmp
    tmp = np.ones([2,(xmax-xmin),3]) * 0
    img[(ymin-2):ymin, xmin:xmax, 0:3] = tmp
    tmp = np.ones([(ymax-ymin),2,3]) * 0
    img[ymin:ymax, (xmax-2):xmax, 0:3] = tmp
    tmp = np.ones([2,(xmax-xmin),3]) * 0
    img[(ymax-2):ymax, xmin:xmax, 0:3] = tmp

    cv2.putText(img, type, (xmin + 10, ymin + 15),
							cv2.FONT_HERSHEY_SIMPLEX, 1e-3 * img.shape[0], color, 2)
    return 

In [4]:
# Crop the image given the coordinates of the cell
def blood_seg_new(xmin, ymin, xmax, ymax, img):
    y, x, c = img.shape
    img[0:ymin, :, :] = np.zeros([ymin, x, 3])
    img[ymax:, :, :] = np.zeros([y-ymax, x, 3])
    img[:, 0:xmin, :] = np.zeros([y, xmin, 3])
    img[:, xmax:, :] = np.zeros([y, x-xmax, 3])
    
    return 

In [5]:
#specify the csv path containing identifiers
csv_path = 'https://raw.githubusercontent.com/Shenggan/BCCD_Dataset/master/test.csv'
#specify root folder for blood images
root = 'https://raw.githubusercontent.com/Shenggan/BCCD_Dataset/master/BCCD/JPEGImages/'
#specify name of the input file

In [6]:
# Read annotations
df = pd.read_csv(csv_path)

In [13]:
# Read the image
blood = "BloodImage_00256.jpg"

rslt_df = df.loc[df['filename'] == blood]
img = get_image(root+blood)

In [14]:
celltypes = ["RBC", "WBC", "Platelets"]
colors = [(0,0,255),(255,0,0),(0,0,0)]

ann_img = img.copy()

for i,name in enumerate(celltypes):
    rslt_df1 = rslt_df.loc[rslt_df['cell_type'] == name]

    for ind in rslt_df1.index:
        xmin = int(rslt_df1["xmin"][ind])
        xmax = int(rslt_df1["xmax"][ind])
        ymin = int(rslt_df1["ymin"][ind])
        ymax = int(rslt_df1["ymax"][ind])
        smear(xmin,ymin,xmax,ymax,ann_img,type=name,color=colors[i])

plt.imshow(ann_img)
plt.title('Blood cells marked using annotations')

In [15]:
# Crop the image around a single WBC
type = 'WBC'
rslt_df1 = rslt_df.loc[rslt_df['cell_type'] == type]

crp_img = img.copy()

ind = rslt_df1.index[0]
xmin = int(rslt_df1["xmin"][ind])
xmax = int(rslt_df1["xmax"][ind])
ymin = int(rslt_df1["ymin"][ind])
ymax = int(rslt_df1["ymax"][ind])

blood_seg_new(xmin, ymin, xmax, ymax, crp_img)

hsv_img = cv2.cvtColor(crp_img, cv2.COLOR_BGR2HSV)

fig, axes = plt.subplots(1, 3)
axes[0].imshow(img)
axes[0].set_title('Original Image')
axes[1].imshow(crp_img)
axes[1].set_title('Cropped around ' + type)
axes[2].imshow(hsv_img)
axes[2].set_title('HSV image ')

In [16]:
# Identify blood cell using HSV thresholding
def hsv_filter(img, lower, upper, lower2=None, upper2=None):
    # Create HSV Image and threshold into a range.
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower, upper)
    
    if lower2 is not None:
        mask2 = cv2.inRange(hsv, lower2, upper2)
        mask = mask + mask2
        
    output = cv2.bitwise_and(img,img, mask= mask)
    
    return output

In [17]:
# Set thresholds type = [lower_th, upper_th]
wbc_th = [np.array([120, 120, 200]), np.array([135, 160, 225])]
plt_th = [np.array([110, 55, 180]), np.array([135, 110, 210])]
rbc_th = [np.array([0, 20, 170]), np.array([10, 50, 200]), np.array([150, 20, 170]), np.array([179, 50, 200])]
                                           
ths = [wbc_th, rbc_th, plt_th]
types = ['WBC', 'RBC', 'Platelet']

In [18]:
fig, axes = plt.subplots(2, 2)
axes[0, 0].imshow(img)
axes[0, 0].set_title('Original')
axes[0, 0].set_xticks([])
axes[0, 0].set_yticks([])

tmp = [axes[0, 1], axes[1, 0], axes[1, 1]]
imgs = []

for i, celltype in enumerate(types):
    tmp_img = img.copy()
    
    if celltype == 'RBC':
        flt_img = hsv_filter(tmp_img, ths[i][0], ths[i][1], ths[i][2], ths[i][3])
    else:
        flt_img = hsv_filter(tmp_img, ths[i][0], ths[i][1])
    
    tmp[i - 1].imshow(flt_img)
    tmp[i - 1].set_title(celltype + ' filtered')
    tmp[i - 1].set_xticks([])
    tmp[i - 1].set_yticks([])
    
    imgs.append(flt_img)

In [19]:
# Otsu's binary thresholding
# Adapted from https://learnopencv.com/otsu-thresholding-with-opencv/
blur = cv2.GaussianBlur(imgs[0], (5, 5), 0)
h, s, v = cv2.split(blur)
#h, s, v = cv2.split(imgs[0])

# Applying Otsu's method setting the flag value into cv.THRESH_OTSU.
# Use a bimodal image as an input.
# Optimal threshold value is cdetermined automatically.
otsu_th, image_result = cv2.threshold(
    v, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU,
)

#blur = cv2.GaussianBlur(image_result, (5, 5), 0)

plt.imshow(image_result)



In [20]:
params = cv2.SimpleBlobDetector_Params() 
# Change thresholds
params.minThreshold = 10
#params.maxThreshold = 200

# Filter by Area.
params.filterByArea = True
#params.minArea = 350
#params.maxArea = 500
params.minArea = 8000
params.maxArea = 60000


# Filter by Circularity
params.filterByCircularity = False
params.minCircularity = 0.1

# Filter by Convexity
params.filterByConvexity = False
params.minConvexity = 0.87

# Filter by Inertia
params.filterByInertia = False
params.minInertiaRatio = 0.01


detector = cv2.SimpleBlobDetector_create(params)

keypoints = detector.detect(cv2.bitwise_not(image_result))
im_with_keypoints = cv2.drawKeypoints(image_result, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

plt.imshow(im_with_keypoints)